# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [53]:
import requests
import os
import json

from IPython.display import JSON

In [ ]:
# import packages we need (remember what packages we used yesterday during the API session)
# IMPORT HERE

## Demo

In [2]:
# don't forget package os
# app_id = os.environ["<your app id name>"]
app_key = os.environ["MiniProject1_TFL_Key"]
url_append = f'?app_key={app_key}' 

In [4]:
# print(url_append)

In [5]:
# URL
url = "https://api.tfl.gov.uk/AirQuality"

In [6]:
# We send the request to the API
# NOTE: if you don't have your APP_KEY, run this without the url_append
res = requests.get(url+url_append)

In [7]:
# We can check if the request was successful
res.status_code

200

In [54]:
# We can export the information that was returned using method .json()
JSON(res.json())

<IPython.core.display.JSON object>

## Define functions for requesting data from TFL api

In [84]:
def get_data(url):
    
    # Get app key
    app_key = os.environ["MiniProject1_TFL_Key"]
    url_append = f'?app_key={app_key}'
    
    # Send request
    res = requests.get(url+url_append)
    
    status_code = res.status_code
    print(status_code)
    
    if status_code >= 200 and status_code < 400:
        return res.json() # Return json format data when the request is successful

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [55]:
url = "https://api.tfl.gov.uk/AirQuality" # Construct url

airquality = get_data(url) # Call function to fetch data

JSON(airquality) # Check the data to see if the request was successful and also be ready to parse it

<IPython.core.display.JSON object>

In [30]:
airquality['currentForecast'][1]['forecastText'] # Parse the data and show the prediction for tomorrow(future)

'Low pressure fronts arriving from the Atlantic will affect the weather in Greater London. Staying mild, cloudy and generally breezy. Spells of rain likely, and could turn heavy at times. &lt;br/&gt;There&#39;ll be a fresh breeze to disperse local emissions and little imported pollution. &lt;br/&gt;&lt;br/&gt;Air pollution is expected to remain &#39;Low&#39; throughout the forecast period for the following pollutants:&lt;br/&gt;&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;Ozone&lt;br/&gt;Sulphur Dioxide &lt;br/&gt;PM10 Particulates&lt;br/&gt;PM2.5 Particulates'

## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [56]:
url = "https://api.tfl.gov.uk/Line/Meta/Modes" # Construct url

modes = get_data(url) # Fetch data

JSON(modes) # Check it

<IPython.core.display.JSON object>

In [35]:
modenames = []
for mode in modes:
    name = mode['modeName'] # Find the modeName field
    if name not in modenames:
        modenames.append(name)
        
print(modenames)
print(f'\nNumber of different modes of transport is {len(modenames)}')

['bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'interchange-keep-sitting', 'interchange-secure', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']

Number of different modes of transport is 18


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [57]:
url = "https://api.tfl.gov.uk/BikePoint"

bikepoints = get_data(url)

JSON(bikepoints)

<IPython.core.display.JSON object>

In [43]:
print(f'{len(bikepoints)} BikePoints in London are operated by Transfor for London')

781 BikePoints in London are operated by Transfor for London


In [44]:
nb_docks = sum([int(point['additionalProperties'][8]['value']) for point in bikepoints])    # Parse and sum for the number of docks for each bikepoint

print(f'{nb_docks} docks are in ALL BikePoints')

20672 docks are in ALL BikePoints


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [58]:
url = 'https://api.tfl.gov.uk/Line/Mode/tube,bus'

lines_tube_bus = get_data(url)

JSON(lines_tube_bus)

<IPython.core.display.JSON object>

In [52]:
print(f'{len(lines_tube_bus)} tube and bus lines are in London!')

687 tube and bus lines are in London!


In [64]:
tube_names = []
for line in lines_tube_bus:
    if line['modeName'] == 'tube':
        tube_names.append(line['name'])

tube_names # List names for all tube lines

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'Hammersmith & City',
 'Jubilee',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'Victoria',
 'Waterloo & City']

## Task
How many station has `victoria` line?

In [68]:
url = 'https://api.tfl.gov.uk/Line/victoria/StopPoints'

victoria_stations = get_data(url)

JSON(victoria_stations)

<IPython.core.display.JSON object>

In [69]:
print(f'Line "Victoria" has {len(victoria_stations)} stations!')

Line "Victoria" has 16 stations!


In [126]:
station_names = []
for station in victoria_stations:
    station_names.append(station['commonName'])
station_names

['Blackhorse Road Underground Station',
 'Brixton Underground Station',
 'Euston Underground Station',
 'Finsbury Park Underground Station',
 'Green Park Underground Station',
 'Highbury & Islington Underground Station',
 "King's Cross St. Pancras Underground Station",
 'Oxford Circus Underground Station',
 'Pimlico Underground Station',
 'Stockwell Underground Station',
 'Seven Sisters Underground Station',
 'Tottenham Hale Underground Station',
 'Victoria Underground Station',
 'Vauxhall Underground Station',
 'Warren Street Underground Station',
 'Walthamstow Central Underground Station']

## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [106]:
origin_keyword = 'Heathrow Airport'
destination_keyword = 'Tower Bridge'
modes = 'bus,tube'

In [95]:
# Get all ICS stopid for StopPoints that containing the keyword for origin

url = f'https://api.tfl.gov.uk/StopPoint/Search?query={origin_keyword}&'
origin_stoppoint = get_data(url)
origin_icsid_name = {}                                    # Key is ICSid, value is name
for place in origin_stoppoint['matches']:
    origin_icsid_name[place['icsId']] = place['name']

origin_icsid_name

200


{'1000104': 'Heathrow Airport Terminal 4',
 '1016430': 'Heathrow Airport Terminal 5'}

In [96]:
# Get all ICS stopid for StopPoints that containing the keyword for destination

url = f'https://api.tfl.gov.uk/StopPoint/Search?query={destination_keyword}&'
destination_stoppoint = get_data(url)
destination_icsid_name = {}                                    # Key is ICSid, value is name
for place in destination_stoppoint['matches']:
    destination_icsid_name[place['icsId']] = place['name']

destination_icsid_name

200


{'1013744': 'Tower Bridge / City Hall',
 '1013746': 'Tower Bridge Road',
 '1019012': "Bricklayer's Arms / Tower Bridge Road",
 '1002087': 'Tower Bridge Quay'}

In [102]:
def find_duration(origin, destination, modes, journey):
    duration_info = {}
    
    duration_info['Origin'] = origin
    duration_info['Destination'] = destination
    duration_info['Total duration'] = journey['duration']
    
    for mode in modes.split(','):
        duration_info[mode] = 0
    
    for leg in journey['legs']:
        duration_info[leg['mode']['name']] += leg['duration']
        
#     for key, item in duration_info.items():
#         print(f'{key}: {item}')
        
    return duration_info

In [115]:
%%time
durations_all = []

for origin in origin_icsid_name.keys():
    for destination in destination_icsid_name.keys():
        
        url = f'https://api.tfl.gov.uk/Journey/JourneyResults/{origin}/to/{destination}?mode={modes}&'
        journeys = get_data(url)
        for journey in journeys['journeys']:
            durations_all.append(find_duration(origin_icsid_name[origin], destination_icsid_name[destination], modes+',walking', journey))
        
sorted_durations_all = sorted(durations_all, key=lambda d: d['Total duration'])
sorted_durations_all

200
200
200
200
200
200
200
200
Wall time: 16.2 s


[{'Origin': 'Heathrow Airport Terminal 5',
  'Destination': 'Tower Bridge / City Hall',
  'Total duration': 76,
  'bus': 5,
  'tube': 59,
  'walking': 0},
 {'Origin': 'Heathrow Airport Terminal 4',
  'Destination': 'Tower Bridge / City Hall',
  'Total duration': 80,
  'bus': 11,
  'tube': 53,
  'walking': 0},
 {'Origin': 'Heathrow Airport Terminal 4',
  'Destination': "Bricklayer's Arms / Tower Bridge Road",
  'Total duration': 99,
  'bus': 86,
  'tube': 0,
  'walking': 0},
 {'Origin': 'Heathrow Airport Terminal 5',
  'Destination': "Bricklayer's Arms / Tower Bridge Road",
  'Total duration': 99,
  'bus': 88,
  'tube': 0,
  'walking': 0},
 {'Origin': 'Heathrow Airport Terminal 5',
  'Destination': "Bricklayer's Arms / Tower Bridge Road",
  'Total duration': 100,
  'bus': 89,
  'tube': 0,
  'walking': 0},
 {'Origin': 'Heathrow Airport Terminal 5',
  'Destination': "Bricklayer's Arms / Tower Bridge Road",
  'Total duration': 102,
  'bus': 91,
  'tube': 0,
  'walking': 0},
 {'Origin': 'He

In [125]:
print(f'The fastest journey to go from {origin_keyword} to {destination_keyword} is:\n')

fastest_journey = sorted_durations_all[0]
keys = list(fastest_journey.keys())

print(f"Origin:\t{fastest_journey['Origin']}")
print(f"Destination:\t{fastest_journey['Destination']}")

keys.remove('Origin')
keys.remove('Destination')

for key in keys:
    print(f'{key} duration:\t{fastest_journey[key]} minutes')

The fastest journey to go from Heathrow Airport to Tower Bridge is:

Origin:	Heathrow Airport Terminal 5
Destination:	Tower Bridge / City Hall
Total duration duration:	76 minutes
bus duration:	5 minutes
tube duration:	59 minutes
walking duration:	0 minutes
